# Supervisor 패턴: 에이전트 관리자

이 노트북에서는 **Supervisor 패턴**을 사용하여 **여러 에이전트를 관리**하는 시스템을 만듭니다.

## Supervisor란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    Supervisor 패턴                                 │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   Supervisor = 여러 전문 에이전트를 관리하는 관리자                │
│                                                                    │
│                    ┌──────────────┐                                │
│                    │  Supervisor  │  ← 누가 일할지 결정            │
│                    └──────┬───────┘                                │
│                           │                                        │
│              ┌────────────┼────────────┐                           │
│              │            │            │                           │
│              ▼            ▼            ▼                           │
│         ┌────────┐  ┌────────┐  ┌────────┐                         │
│         │Researcher│  │ Coder │  │ Writer │  ← 전문 에이전트       │
│         └────────┘  └────────┘  └────────┘                         │
│                                                                    │
│   역할:                                                            │
│   • Supervisor: 작업 분배, 진행 상황 모니터링, 완료 판단           │
│   • 에이전트: 각자 전문 분야의 작업 수행                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 아키텍처

```
┌────────────────────────────────────────────────────────────────────┐
│                    Supervisor 그래프 구조                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   START                                                            │
│     │                                                              │
│     ▼                                                              │
│   ┌──────────────┐                                                 │
│   │  supervisor  │ ◄─────────────────────┐                         │
│   └──────┬───────┘                       │                         │
│          │                               │                         │
│    next = ?                              │                         │
│    ╱    │    ╲                           │                         │
│   ╱     │     ╲                          │                         │
│  ▼      ▼      ▼                         │                         │
│ researcher  coder  FINISH                │                         │
│  │       │      │                        │                         │
│  │       │      └────────► END           │                         │
│  │       │                               │                         │
│  └───────┴───────────────────────────────┘                         │
│          (결과 보고)                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. Supervisor 결정 스키마 정의

Supervisor가 다음에 어떤 에이전트를 호출할지 결정하는 구조를 정의합니다.

In [ ]:
from typing import Literal
from pydantic import BaseModel

class SupervisorDecision(BaseModel):
    """
    Supervisor의 결정 스키마
    
    next: 다음에 호출할 에이전트 또는 종료 신호
    """
    next: Literal['researcher', 'coder', 'FINISH']

print("✅ SupervisorDecision 스키마 정의 완료")
print("   가능한 값: researcher, coder, FINISH")

# 3. LLM 설정 (Structured Output)

In [ ]:
from langchain_ollama import ChatOllama

# 기본 모델
base_model = ChatOllama(model='llama3.2', temperature=0)

# Structured Output으로 변환 (Supervisor용)
model = base_model.with_structured_output(SupervisorDecision)

print("✅ LLM 설정 완료")
print("   with_structured_output: 항상 SupervisorDecision 형태로 응답")

# 4. 에이전트 정의

In [ ]:
# 사용 가능한 에이전트 목록
agents = ['researcher', 'coder']

print("✅ 에이전트 목록 정의")
print(f"   에이전트: {agents}")

# 5. State 정의

In [ ]:
from langgraph.graph import MessagesState

class AgentState(MessagesState):
    """
    에이전트 상태
    
    messages: 대화 기록 (MessagesState에서 상속)
    next: 다음에 호출할 에이전트
    """
    next: Literal['researcher', 'coder', 'FINISH']

print("✅ AgentState 정의 완료")

# 6. Supervisor 프롬프트 및 노드 정의

In [ ]:
# Supervisor 시스템 프롬프트
system_prompt_part_1 = f'''당신은 다음 서브에이전트 사이의 대화를 관리하는 슈퍼바이저입니다. 
서브에이전트: {agents}. 
아래 사용자 요청에 따라, 다음으로 행동할 서브에이전트를 지목하세요. 
각 서브에이전트는 임무를 수행하고 결과와 상태를 응답합니다. 
실행할 서브에이전트가 없거나 작업이 완료되면, FINISH로 응답하세요.'''

system_prompt_part_2 = f'''위 대화를 바탕으로, 다음으로 행동할 서브에이전트는 누구입니까? 
아니면 FINISH 해야 합니까? 
서브에이전트: {", ".join(agents)}, FINISH'''


def supervisor(state: AgentState):
    """
    Supervisor 노드
    
    1. 현재 대화 상태 분석
    2. 다음 에이전트 결정 (또는 FINISH)
    3. 결정 반환
    """
    messages = [
        ('system', system_prompt_part_1),
        *state['messages'],
        ('system', system_prompt_part_2),
    ]
    
    # LLM이 SupervisorDecision 형태로 응답
    decision = model.invoke(messages)
    
    return decision  # {'next': 'researcher'} 또는 {'next': 'FINISH'} 등


print("✅ Supervisor 노드 정의 완료")

# 7. 에이전트 노드 정의

In [ ]:
def researcher(state: AgentState):
    """
    Researcher 에이전트
    
    데이터 검색 및 조사 담당
    """
    response = {
        'role': 'assistant',
        'content': '관련 데이터를 찾는 중입니다... 잠시만 기다려주세요.',
    }
    
    # 실제로는 검색 도구 등을 사용
    # 여기서는 시뮬레이션
    fake_data = {
        'data': '전세계 인구 데이터: [미국: 331M, 중국: 1.4B, 인도: 1.3B]'
    }
    response['content'] += f"\n찾은 데이터: {fake_data['data']}"
    
    return {'messages': [response]}


def coder(state: AgentState):
    """
    Coder 에이전트
    
    코드 작성 담당
    """
    response = {
        'role': 'assistant',
        'content': '코드를 작성 중입니다... 잠시만 기다려주세요.',
    }
    
    # 실제로는 코드 생성 LLM 등을 사용
    # 여기서는 시뮬레이션
    fake_code = '''
def visualize_population(data):
    import matplotlib.pyplot as plt

    countries = list(data.keys())
    population = list(data.values())

    plt.bar(countries, population)
    plt.xlabel('Country')
    plt.ylabel('Population')
    plt.title('World Population by Country')
    plt.show()

data = {'USA': 331, 'China': 1400, 'India': 1300}
visualize_population(data)
'''
    response['content'] += f"\n작성된 코드:\n{fake_code}"
    
    return {'messages': [response]}


print("✅ 에이전트 노드 정의 완료")
print("   - researcher: 데이터 검색")
print("   - coder: 코드 작성")

# 8. 그래프 구성

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)

# 노드 추가
builder.add_node('supervisor', supervisor)
builder.add_node('researcher', researcher)
builder.add_node('coder', coder)

# 엣지 추가
builder.add_edge(START, 'supervisor')  # 시작 → Supervisor

# Supervisor의 결정에 따라 라우팅
builder.add_conditional_edges(
    'supervisor',
    lambda state: state['next'],  # next 값으로 라우팅
    {
        'researcher': 'researcher',
        'coder': 'coder',
        'FINISH': END,
    }
)

# 에이전트 → Supervisor (결과 보고)
builder.add_edge('researcher', 'supervisor')
builder.add_edge('coder', 'supervisor')

graph = builder.compile()

print("✅ 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== Supervisor 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 9. Supervisor 에이전트 실행

In [ ]:
# 테스트 실행
initial_state = {
    'messages': [
        {
            'role': 'user',
            'content': '전세계 인구를 국적을 기준으로 시각화 해주세요.',
        }
    ],
    'next': 'supervisor',
}

print("=== Supervisor 에이전트 실행 ===")
print(f"사용자 요청: {initial_state['messages'][0]['content']}\n")

for output in graph.stream(initial_state):
    node_name, node_result = list(output.items())[0]
    
    print(f"\n{'─'*60}")
    print(f"현재 노드: {node_name}")
    
    if node_result.get('messages'):
        content = node_result['messages'][-1]
        if isinstance(content, dict):
            print(f"응답: {content['content'][:200]}..." if len(content['content']) > 200 else f"응답: {content['content']}")
        else:
            print(f"응답: {str(content)[:200]}...")
    
    if node_result.get('next'):
        print(f"다음 단계: {node_result['next']}")

---

## 정리: Supervisor 패턴

### 핵심 개념

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Supervisor 패턴 동작 원리                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 사용자 요청 접수                                               │
│   2. Supervisor가 적절한 에이전트 선택                              │
│   3. 선택된 에이전트가 작업 수행                                    │
│   4. 결과를 Supervisor에게 보고                                     │
│   5. Supervisor가 다음 에이전트 선택 (또는 FINISH)                  │
│   6. 반복...                                                        │
│                                                                     │
│   핵심: Supervisor가 작업 흐름을 제어!                              │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 결정 스키마
class SupervisorDecision(BaseModel):
    next: Literal['agent1', 'agent2', 'FINISH']

# 2. Structured Output LLM
model = base_model.with_structured_output(SupervisorDecision)

# 3. 조건부 라우팅
builder.add_conditional_edges(
    'supervisor',
    lambda state: state['next'],
    {'agent1': 'agent1', 'agent2': 'agent2', 'FINISH': END}
)

# 4. 에이전트 → Supervisor 연결
builder.add_edge('agent1', 'supervisor')
builder.add_edge('agent2', 'supervisor')
```

### Supervisor vs 다른 패턴

| 패턴 | 제어 방식 | 적합한 상황 |
|------|----------|------------|
| **Supervisor** | 중앙 집중 | 작업 순서가 중요, 복잡한 조정 |
| **Subgraph** | 계층적 | 모듈화, 재사용 |
| **Swarm** | 분산 | 병렬 처리, 독립적 작업 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
model = model.with_structured_output(SupervisorDecision)

# 변경
base_model = ChatOllama(model='llama3.2', temperature=0)
model = base_model.with_structured_output(SupervisorDecision)
```

## ch07 요약: 고급 에이전트 패턴

| 패턴 | 용도 | 핵심 기술 |
|------|------|----------|
| **Reflection** | 자기 개선 | 역할 변환 트릭 |
| **Subgraph (직접)** | 그래프 모듈화 | 상태 키 공유 |
| **Subgraph (함수)** | 상태 변환 | 래퍼 함수 |
| **Supervisor** | 에이전트 관리 | Structured Output |

## 다음 단계

**ch08에서는** Structured Output, Streaming, Interrupt 등 **고급 기능**을 배웁니다.